<a href="https://colab.research.google.com/github/talhaanwarch/OffenseEval2020/blob/master/Danish/sub/sub1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Danish Language


In [1]:
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

Mounted at /content/drive


In [2]:
!pip install focal-loss
!pip install keras-radam

In [3]:
cd /content/drive/My Drive/dataset/OffenseEval2020/data/Danish/

/content/drive/My Drive/dataset/OffenseEval2020/data/Danish


In [4]:
ls

cc.da.300.vec     offenseval-da-test-v1-nolabels.tsv
cc.da.300.vec.gz  offenseval-da-training-v1.tsv


In [0]:
import pandas as pd
train=pd.read_csv( 'offenseval-da-training-v1.tsv',sep="\t")
train.drop(train.tail(n=1).index,inplace=True)
train["tweet"]= train["tweet"].str.replace('@USER', "") 
train['tweet']=train['tweet'].str.replace('\d+', '')
train['tweet']=train['tweet'].str.replace('URL', '')
train["tweet"]= train["tweet"].str.lower()
train=train.sample(frac=1)

In [6]:
train.head()

,id,tweet,subtask_a
1758,949,"nu er det efterhånden mange år siden, men jeg ...",NOT
867,2109,"altså det skal ikke være stearinlys, men nogle...",NOT
1035,1370,enig det er vore laden stå til moral,NOT
2555,3361,"kongen er død, længe leve kong frederik.",NOT
1791,1050,de kan godt forstå regionerne mangler million...,NOT


In [7]:
train.tail()

,id,tweet,subtask_a
2923,243,hvordan i alverden kan der gå næsten et år før...,NOT
158,199,dronningen er altså ikke syg,NOT
2805,17,hvem er klar på at tune én og fyre en md-kasse...,NOT
1103,2374,det värsta är att det är sant också! danskjävlar!,OFF
1608,3439,"jeg ville ønske, migmiger brugte grammatisk ko...",NOT


In [0]:
labels=train['subtask_a']
train=train['tweet']
from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()
labels=le.fit_transform(labels)

In [9]:
import collections
collections.Counter(labels)

Counter({0: 2576, 1: 384})

In [10]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
max_words = 10000 #frequency of words to be kept
max_len = 200

tokenize = Tokenizer(num_words=max_words)
tokenize.fit_on_texts(train)
sequences = tokenize.texts_to_sequences(train)
word_index = tokenize.word_index
sequences_matrix = sequence.pad_sequences(sequences,maxlen=max_len)

Using TensorFlow backend.


In [11]:
len(sequences_matrix),len(labels)

(2960, 2960)

In [0]:
from keras.models import Sequential
from keras.layers import Embedding,CuDNNGRU,GRU,Dense,Dropout,Bidirectional,GlobalMaxPool1D,GlobalAveragePooling1D, SpatialDropout1D
from keras.optimizers import RMSprop
from keras.initializers import Constant
from sklearn.utils import class_weight
from sklearn.metrics import classification_report,f1_score
import keras
from keras_radam import RAdam
import numpy as np
from sklearn.model_selection import StratifiedKFold
from focal_loss import BinaryFocalLoss
import tensorflow as tf
tf.logging.set_verbosity(tf.logging.ERROR)

In [13]:
num_words = min(max_words, len(word_index)) + 1
print(num_words)

9802


In [0]:
# !wget https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.el.300.vec.gz

In [0]:
# import gzip
# import shutil
# with gzip.open('cc.el.300.vec.gz', 'rb') as f_in:
#     with open('cc.el.300.vec', 'wb') as f_out:
#         shutil.copyfileobj(f_in, f_out)

In [0]:
embed_size=300
def get_coefs(word,*arr):
    return word, np.asarray(arr, dtype='float32')

def build_matrix(embedding_path, word_index):
    embedding_index = dict(get_coefs(*o.strip().split(" ")) for o in open(embedding_path))

    nb_words = min(max_words, len(word_index))
    embedding_matrix = np.zeros((nb_words + 1, embed_size))
    for word, i in word_index.items():
        if i >= max_words:
            continue
        embedding_vector = embedding_index.get(word)
        if embedding_vector is not None:
            embedding_matrix[i] = embedding_vector
    return embedding_matrix

In [0]:
embeddings=build_matrix('cc.da.300.vec', word_index)


In [18]:
embeddings.shape

(9802, 300)

In [0]:
def model_func():
  model = Sequential()
  model.add(Embedding(num_words,embed_size,embeddings_initializer=Constant(embeddings),input_length=max_len,trainable=False))
  model.add(Bidirectional(CuDNNGRU(150, return_sequences = True)))
  model.add((CuDNNGRU(32, return_sequences = True)))
  model.add(GlobalMaxPool1D())
  model.add(Dense(100, activation="relu"))
  model.add(Dense(50, activation="relu"))
  #model.add(Dense(10, activation="relu"))

  model.add(Dropout(0.05))
  model.add(Dense(1, activation="sigmoid"))
  model.compile(loss=BinaryFocalLoss(gamma=2), optimizer='adam', metrics=['accuracy'])
  return model

In [0]:
import numpy as np
class_weights = class_weight.compute_class_weight('balanced',np.unique(labels),labels)
class_weights=dict(enumerate(class_weights))

In [0]:
from keras.callbacks import ReduceLROnPlateau
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2,
                              patience=2, min_lr=0.000001, verbose=1)

In [30]:
from sklearn.model_selection import train_test_split
x_train,x_val,y_train,y_val=train_test_split(sequences_matrix,labels,test_size=0.15,stratify=labels,shuffle=True)
model=model_func()
model.fit(x_train,y_train,batch_size=64,epochs=10,verbose=2,class_weight=class_weights,validation_data=(x_val,y_val))

Train on 2516 samples, validate on 444 samples
Epoch 1/10
 - 3s - loss: 0.1172 - acc: 0.8541 - val_loss: 0.1078 - val_acc: 0.8694
Epoch 2/10
 - 2s - loss: 0.1049 - acc: 0.8704 - val_loss: 0.0976 - val_acc: 0.8694
Epoch 3/10
 - 2s - loss: 0.0881 - acc: 0.8812 - val_loss: 0.0862 - val_acc: 0.8896
Epoch 4/10
 - 2s - loss: 0.0733 - acc: 0.9086 - val_loss: 0.0762 - val_acc: 0.9032
Epoch 5/10
 - 2s - loss: 0.0666 - acc: 0.9141 - val_loss: 0.0742 - val_acc: 0.9122
Epoch 6/10
 - 2s - loss: 0.0617 - acc: 0.9169 - val_loss: 0.0759 - val_acc: 0.9054
Epoch 7/10
 - 2s - loss: 0.0557 - acc: 0.9308 - val_loss: 0.0821 - val_acc: 0.9167
Epoch 8/10
 - 2s - loss: 0.0498 - acc: 0.9356 - val_loss: 0.0862 - val_acc: 0.8829
Epoch 9/10
 - 2s - loss: 0.0489 - acc: 0.9436 - val_loss: 0.0902 - val_acc: 0.9054
Epoch 10/10
 - 2s - loss: 0.0386 - acc: 0.9547 - val_loss: 0.0959 - val_acc: 0.9032


In [31]:
from sklearn.metrics import classification_report

y_pred = model.predict(x_val, batch_size=64, verbose=1)
y_pred_bool = np.argmax(y_pred, axis=1)
y_pred = (y_pred > 0.45)

print(classification_report(y_val, y_pred))

444/444 [==============================] - 0s 492us/step
              precision    recall  f1-score   support

           0       0.92      0.96      0.94       386
           1       0.66      0.47      0.55        58

    accuracy                           0.90       444
   macro avg       0.79      0.71      0.74       444
weighted avg       0.89      0.90      0.89       444



In [24]:
test=pd.read_csv( 'offenseval-da-test-v1-nolabels.tsv',sep='\t')
test["tweet"]= test["tweet   subtask_a"].str.replace('@USER', "") 
test['tweet']=test['tweet'].str.replace('\d+', '')
test['tweet']=test['tweet'].str.replace('URL', '')
test['tweet']=test['tweet'].str.lower()

test.head()

,id,tweet subtask_a,tweet
0,1382,"Der er syriske ""flygtninge"" som rejser til Ira...","der er syriske ""flygtninge"" som rejser til ira..."
1,1384,Danmark = Vitryssland?,danmark = vitryssland?
2,547,Ja tvangsfjernelser af børn på urigtige oplysn...,ja tvangsfjernelser af børn på urigtige oplysn...
3,1269,Han kan ikke Svensk og forventer et job. Hvis ...,han kan ikke svensk og forventer et job. hvis ...
4,1695,NED MED SVENSKEN!,ned med svensken!


In [0]:
X_test=test['tweet']

test_sequences = tokenize.texts_to_sequences(X_test)
X_test_sequences = sequence.pad_sequences(test_sequences,maxlen=max_len)

In [26]:
y_test = model.predict(X_test_sequences, batch_size=64, verbose=1)
y_test[y_test>0.5]=1
y_test[y_test<0.5]=0
y_test=y_test.astype('int16').ravel()

329/329 [==============================] - 0s 262us/step


In [27]:
y_test=le.inverse_transform(y_test)
y_test=pd.DataFrame(y_test,columns=['label'])
y_test=pd.concat([test['id'], y_test['label']], axis=1)
y_test.head()

,id,label
0,1382,NOT
1,1384,NOT
2,547,NOT
3,1269,NOT
4,1695,NOT


In [0]:
y_test.to_csv('/content/sub1.csv',index=False,header=None)


In [0]:
import keras
keras.backend.clear_session()